# Keras VGG16 model to solve the Pneumonia problem

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, MaxPool2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

### Fix some tensorflow and cudNN incompatibilities

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

### Setting the TF warnings to low

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Getting the data

In [4]:
batch_size = 32
IMG_SIZE = 150

classes = ['NORMAL', 'PNEUMONIA']
train_path = '../../Datasets/chest_xray/train'
test_path = '../../Datasets/chest_xray/test'

data_gen = ImageDataGenerator(rescale=1./255)
train_batches = data_gen.flow_from_directory(train_path, target_size=(IMG_SIZE, IMG_SIZE), classes=classes, 
                                             class_mode='binary', batch_size=batch_size)

test_batches = data_gen.flow_from_directory(test_path, target_size=(IMG_SIZE, IMG_SIZE), classes=classes, 
                                            class_mode='binary')

Found 21216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


## Create the model

### Hiperparams:

In [5]:
lr = 0.0001
beta_1 = 0.9
beta_2 = 0.999
epochs = 5
steps_per_epoch = 663
validation_steps = 624
DROP_PROB = 0.4

### VGG16

In [6]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=train_batches.image_shape))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=(2, 2)))
model.add(Dropout(DROP_PROB))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=(2, 2)))
model.add(Dropout(DROP_PROB))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=(2, 2)))
model.add(Dropout(DROP_PROB))

model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=(2, 2)))
model.add(Dropout(DROP_PROB))

model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=(2, 2)))
model.add(Dropout(DROP_PROB))

model.add(Flatten())

model.add(Dense(4090, activation='relu'))
model.add(Dropout(DROP_PROB))
model.add(BatchNormalization())
model.add(Dense(4090, activation='relu'))
model.add(Dropout(DROP_PROB))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 64)      256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)       1

### Compiling the net

In [8]:
optimizer = Adam(lr=lr, beta_1=beta_1, beta_2=beta_2)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [9]:
history = model.fit_generator(epochs=5, shuffle=True, generator=train_batches,
        validation_data=test_batches, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps)

Epoch 1/5
663/663 [==============================] - 331s 499ms/step - loss: 0.3085 - accuracy: 0.8878 - val_loss: 5.7428 - val_accuracy: 0.6251
Epoch 2/5
663/663 [==============================] - 335s 506ms/step - loss: 0.1617 - accuracy: 0.9460 - val_loss: 5.1600 - val_accuracy: 0.6248
Epoch 3/5
663/663 [==============================] - 332s 500ms/step - loss: 0.1185 - accuracy: 0.9570 - val_loss: 5.0858 - val_accuracy: 0.6254
Epoch 4/5
663/663 [==============================] - 335s 505ms/step - loss: 0.1005 - accuracy: 0.9656 - val_loss: 3.1998 - val_accuracy: 0.6377
Epoch 5/5
663/663 [==============================] - 335s 505ms/step - loss: 0.0861 - accuracy: 0.9694 - val_loss: 4.5473 - val_accuracy: 0.6248
